## 데이터 및 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import pandas as pd 
import geopandas as gpd
import numpy as np
from geopy.distance import geodesic
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import haversine_distances
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# 도로 예측 데이터 불러오기
df = pd.read_csv('../data/화성_도로예측.csv')
df

,id,ALL_AADT,grid_density_mean,buffer_500m_density_mean,buffer_1km_density_mean,pop_avg_100m,pop_avg_500m,pop_avg_1km,min_dist_to_rest,min_dist_to_sub
0,47834686201,27724.0,1.0,1.000000,0.972350,58.996484,88.680012,48.326760,7772.517840,6572.074344
1,47735667801,11939.0,1.0,0.714286,0.716814,37.545854,47.595515,28.651125,6670.115741,5706.176951
2,47825587301,10733.0,1.0,1.142857,1.371094,19.688529,15.010157,12.941629,4571.063813,3886.564032
3,47735208501,28034.0,0.0,0.000000,0.000000,18.698021,25.582787,11.286719,8499.691329,3018.999790
4,47825020201,6165.0,1.0,0.971014,1.042194,22.964521,67.429104,36.040223,6598.762094,2066.172807
...,...,...,...,...,...,...,...,...,...,...
1137,47835413601,10641.0,2.0,1.574074,1.355769,19.400462,20.470623,39.068485,4840.615240,4184.699840
1138,47824327901,13332.0,1.0,1.093750,1.356522,13.302683,10.422081,11.435672,4465.706961,3788.754576
1139,47714507401,7604.0,0.0,0.000000,0.000000,18.202419,18.830942,12.786421,13147.172380,16963.808053
1140,47835608401,15383.0,0.2,0.371134,0.422145,0.559468,8.979918,16.118917,5554.617507,2829.946555


In [2]:
# 종속변수 설정
X = df.drop(['id', 'ALL_AADT'], axis=1)
y = df['ALL_AADT']

# 스케일링
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42) 

## 기본 모델링

In [16]:
# 선형 회귀 모델 생성
linear_model = LinearRegression()

# 모델 학습
linear_model.fit(X_train, y_train)

# 예측
y_pred_ratio_linear_1 = linear_model.predict(X_test)

# 평가
mse = mean_squared_error(y_test, y_pred_ratio_linear_1)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_ratio_linear_1)

n = X_test.shape[0] 
k = X_test.shape[1] 
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)

print(f"Linear Regression RMSE: {rmse:.4f}")
print(f"R^2: {r2:.4f}")
print(f"Adjusted R^2: {adjusted_r2:.4f}")

Linear Regression RMSE: 13841.7867
R^2: 0.1566
Adjusted R^2: 0.1259


### 랜덤포레스트리그레서 
- 튜닝해본 결과 R^2 기준 0.7대 초반 전전

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# 랜덤 포레스트 모델 생성
rf = RandomForestRegressor(random_state=42)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2],
    'min_samples_leaf': [1, 2]
}

# GridSearchCV 설정
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# 모델 학습
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)

# 최적 모델로 예측
best_rf = grid_search.best_estimator_
y_pred_ratio_rf = best_rf.predict(X_test)

# 평가
mse = mean_squared_error(y_test, y_pred_ratio_rf)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_ratio_rf)

n = X_test.shape[0]
k = X_test.shape[1]
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)

print(f"Random Forest (Grid Search) RMSE: {rmse:.4f}")
print(f"R^2: {r2:.4f}")
print(f"Adjusted R^2: {adjusted_r2:.4f}")

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.5s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.4s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.5s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.4s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.4s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.9s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.8s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.9s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.8s
[CV] END max_depth=10, min_samples

### LGBMRegressor
- 그리드 서치를 통해 튜닝해본 결과 R^2 기준 0.7 초반대 전전

In [ ]:
from lightgbm import LGBMRegressor

# LightGBM 모델 생성
lgbm = LGBMRegressor(random_state=42)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20, None], 
    'learning_rate': [0.1, 0.2, 0.3],
    'num_leaves': [30, 40, 50],
}

# GridSearchCV 설정
grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# 모델 학습
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)

# 최적 모델로 예측
best_lgbm = grid_search.best_estimator_
y_pred_ratio_lgbm = best_lgbm.predict(X_test)

# 평가
mse = mean_squared_error(y_test, y_pred_ratio_lgbm)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_ratio_lgbm)

n = X_test.shape[0]
k = X_test.shape[1]
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)

print(f"LightGBM (Grid Search) RMSE: {rmse:.4f}")
print(f"R^2: {r2:.4f}")
print(f"Adjusted R^2: {adjusted_r2:.4f}")

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] END learning_rate=0.1, max_depth=10, n_estimators=50, num_leaves=30; total time= 2.3min
[CV] END learning_rate=0.1, max_depth=10, n_estimators=50, num_leaves=30; total time= 2.5min
[CV] END learning_rate=0.1, max_depth=10, n_estimators=50, num_leaves=30; total time= 3.2min
[CV] END learning_rate=0.1, max_depth=10, n_estimators=50, num_leaves=30; total time= 4.4min
[CV] END learning_rate=0.1, max_depth=10, n_estimators=50, num_leaves=30; total time= 2.2min


### Catboost  : best_model
- 'depth': 12, 'iterations': 400, 'learning_rate': 0.2
- R square : 0.7761 

In [5]:
from catboost import CatBoostRegressor

# CatBoost 모델 생성
cb = CatBoostRegressor(random_seed=42, verbose=0)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'iterations': [100, 300, 500],
    'depth': [12, 14],
    'learning_rate': [0.1, 0.3, 0.5],
    'l2_leaf_reg': [1, 3]
}

# GridSearchCV 설정
grid_search = GridSearchCV(estimator=cb, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# 모델 학습
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)

# 최적 모델로 예측
best_cb = grid_search.best_estimator_
y_pred_ratio_cb = best_cb.predict(X_test)

# 평가
mse = mean_squared_error(y_test, y_pred_ratio_cb)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_ratio_cb)

n = X_test.shape[0]
k = X_test.shape[1]
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)

print(f"CatBoost (Grid Search) RMSE: {rmse:.4f}")
print(f"R^2: {r2:.4f}")
print(f"Adjusted R^2: {adjusted_r2:.4f}")

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END ........depth=10, iterations=200, learning_rate=0.1; total time=  15.8s
[CV] END ........depth=10, iterations=200, learning_rate=0.1; total time=  14.7s
[CV] END ........depth=10, iterations=200, learning_rate=0.1; total time=  15.0s
[CV] END .......depth=10, iterations=200, learning_rate=0.15; total time=  14.9s
[CV] END .......depth=10, iterations=200, learning_rate=0.15; total time=  15.1s
[CV] END .......depth=10, iterations=200, learning_rate=0.15; total time=  15.7s
[CV] END ........depth=10, iterations=200, learning_rate=0.2; total time=  15.2s
[CV] END ........depth=10, iterations=200, learning_rate=0.2; total time=  14.9s
[CV] END ........depth=10, iterations=200, learning_rate=0.2; total time=  15.1s
[CV] END ........depth=10, iterations=400, learning_rate=0.1; total time=  29.9s
[CV] END ........depth=10, iterations=400, learning_rate=0.1; total time=  30.3s
[CV] END ........depth=10, iterations=400, learn